# User Notebook

In [1]:
import pandas as pd
import numpy as np
from data_prep_functions import *

In [2]:
a = household_number_comparison_houseolds_and_input_files('39', True)
print('1')
b = household_number_comparison_houseolds_and_input_files('41', True)
print('2')
c = household_number_comparison_houseolds_and_input_files('46', True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


1
2


In [2]:
final = population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=False, to_jdrive=False)
final

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_input_files,pop_count_household_file,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
238574,2050,22996,237,237,0
238575,2050,22998,218,218,0
238576,2050,23000,312,312,0
238577,2050,23001,195,195,0


In [3]:
final = population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=False, to_jdrive=False)
final

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,pop_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
238574,2050,22996,237,237,0
238575,2050,22998,218,218,0
238576,2050,23000,312,312,0
238577,2050,23001,195,195,0


In [4]:
population_comparison_houseolds_and_input_files('39', gq_only=False, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('39', gq_only=True, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('39', gq_only=False, no_gq=True, to_jdrive=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,hhp_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
237715,2050,22996,228,228,0
237716,2050,22998,184,184,0
237717,2050,23000,298,298,0
237718,2050,23001,194,194,0


In [5]:
population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('41', gq_only=True, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('41', gq_only=False, no_gq=True, to_jdrive=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,hhp_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
238293,2050,22996,237,237,0
238294,2050,22998,218,218,0
238295,2050,23000,312,312,0
238296,2050,23001,195,195,0


In [6]:
population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('46', gq_only=True, no_gq=False, to_jdrive=True)
population_comparison_houseolds_and_input_files('46', gq_only=False, no_gq=True, to_jdrive=True)

c:\Users\cra\OneDrive - San Diego Association of Governments\QA_Repository\2022\Forecast_Automation\scripts_2.0\data_prep_functions.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_file_subset['year'] = pd.to_numeric(


,year,mgra,pop_count_household_file,hhp_input_files,Diff
0,2016,1,41,41,0
1,2016,2,81,81,0
2,2016,3,111,111,0
3,2016,4,73,73,0
4,2016,5,63,63,0
...,...,...,...,...,...
237696,2050,22996,236,236,0
237697,2050,22998,196,196,0
237698,2050,23000,307,307,0
237699,2050,23001,184,184,0
